In [ ]:
import pandas as pd
import os
from sklearn.inspection import permutation_importance
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import StandardScaler
from sklearn.svm import SVC
from sklearn.linear_model import LogisticRegression
from xgboost import XGBClassifier
from sklearn.naive_bayes import GaussianNB
from sklearn.model_selection import KFold, GridSearchCV
from sklearn.metrics import accuracy_score, balanced_accuracy_score, precision_score, recall_score, f1_score, roc_curve, auc
import shap
import matplotlib.pyplot as plt
from matplotlib import cm
from mpl_toolkits.mplot3d import Axes3D
import numpy as np
from sklearn.feature_selection import SelectKBest, mutual_info_classif
from skrebate import MultiSURF
import seaborn as sns

In [ ]:
# Define directory path
directory_path = 'data/small_datasets/'
# Change the current working directory
os.chdir(directory_path)
# Confirm the current working directory
print("Current working directory:", os.getcwd())
# Get a list of files and directories
files = os.listdir(directory_path)
# Display the files and directories
print("Files and directories in '", directory_path, "':")
print(files)

In [ ]:
datasets_names = ['4-wayAdditive_100feat.txt', '2Additive_2-wayEpi_100feat.txt', '2-wayEpi_100feat.txt', '4-wayHeterogeneous_100feat.txt']

# Loop through each dataset
for name in datasets_names:
    # Construct the full path to the dataset file
    file_path = os.path.join(directory_path, name)

    # Load the dataset into a DataFrame
    df = pd.read_csv(file_path, delimiter='\t')
    
    # Print name of dataset and first few rows
    print(f'Dataset: {name}')
    print(df.head())
    
    # Print the shape of the DataFrame
    num_instances = df.shape[0]
    num_features = df.shape[1]
    print(f'Number of instances in {name}: {num_instances}')
    print(f'Number of features in {name}: {num_features}')
    print("\n" + "="*80 + "\n")  # Separator for readability


In [ ]:
# Choose the column for which you want to plot the counts
column_name = 'Class'

# Color palette for the bar plots
palette = sns.color_palette("husl", len(datasets_names))

# Loop through each dataset, load it, and create a bar plot
for idx, name in enumerate(datasets_names):
    # Construct the full path to the dataset file
    file_path = directory_path + name
    
    # Load the dataset into a DataFrame
    df = pd.read_csv(file_path, delimiter='\t')
    
    # Get the value counts for the specified column
    value_counts = df[column_name].value_counts()
    
    # Print the distribution of the class
    print(f'Dataset: {name}')
    print(f'Distribution of {column_name}:')
    print(value_counts)
    print(f'Percentage distribution:')
    print(value_counts / value_counts.sum() * 100)
    print("\n" + "="*80 + "\n")  # Separator for readability

    # Plot the value counts using seaborn
    plt.figure(figsize=(6, 4))
    sns.barplot(x=value_counts.index, y=value_counts.values, palette=[palette[idx]], width=0.4)
    plt.title(f'Counts of {column_name} in {name}')
    plt.xlabel(column_name)
    plt.ylabel('Count')
    plt.xticks(rotation=0)
    plt.show()

In [ ]:
# Missing Value Assessment

# Loop through each dataset name
for name in datasets_names:
    
    # Group the DataFrame by the "class" column
    grouped_df = df.groupby('Class')

    # Calculate the sum of missing values in rows within each group
    missing_values_per_class = grouped_df.apply(lambda x: x.isnull().sum().sum())

    # Now 'missing_values_per_class' contains the sum of missing values for each class
    print("Sum of missing values for each class:")
    print(missing_values_per_class)

    # Count missing values in dataset
    total_missing_values = df.isna().sum().sum()

    # Now 'total_missing_values' contains the total number of missing values in the DataFrame
    print("Total number of missing values:", total_missing_values)

In [ ]:
# Define MultiSURF and other options
do_multisurf = True  # (bool) do MultiSURF analysis
njobs = -1  # (int) number of cores dedicated to running algorithm; -1 will use all available cores

# To store ROC results for final plotting
roc_results = {}

# Directory to save plots and feature importances
output_dir = 'results/'
os.makedirs(output_dir, exist_ok=True)

def evaluate_model(model, param_grid, X, y, n_splits=5):
    # Initialize KFold cross-validation
    kf = KFold(n_splits=n_splits, shuffle=True, random_state=42)

    # Initialize GridSearchCV
    grid_search = GridSearchCV(estimator=model, param_grid=param_grid, cv=kf, scoring='accuracy', n_jobs=-1)

    # Fit GridSearchCV
    grid_search.fit(X, y)

    # Best model from GridSearchCV
    best_model = grid_search.best_estimator_

    # Cross-validation results
    cv_results = grid_search.cv_results_

    # Initialize evaluation metrics lists
    accuracies = []
    balanced_accuracies = []
    precisions = []
    recalls = []
    f1_scores = []

    # Initialize lists to store true labels and predicted probabilities
    true_labels = []
    predicted_probabilities = []

    # Iterate over each fold of cross-validation
    for train_index, test_index in kf.split(X):
        # Split data into train and test sets for this fold
        X_train, X_test = X.iloc[train_index], X.iloc[test_index]
        y_train, y_test = y.iloc[train_index], y.iloc[test_index]

        best_model.fit(X_train, y_train)

        # Predict on the testing data
        y_pred = best_model.predict(X_test)

        # Get predicted probabilities for the positive class
        if hasattr(best_model, "predict_proba"):
            y_probs = best_model.predict_proba(X_test)[:, 1]
        else:
            y_probs = best_model.decision_function(X_test)
            y_probs = (y_probs - y_probs.min()) / (y_probs.max() - y_probs.min())

        # Calculate evaluation metrics
        accuracies.append(accuracy_score(y_test, y_pred))
        balanced_accuracies.append(balanced_accuracy_score(y_test, y_pred))
        precisions.append(precision_score(y_test, y_pred))
        recalls.append(recall_score(y_test, y_pred))
        f1_scores.append(f1_score(y_test, y_pred))

        # Append true labels and predicted probabilities
        true_labels.extend(y_test)
        predicted_probabilities.extend(y_probs)

    # Calculate average evaluation metrics
    avg_accuracy = sum(accuracies) / len(accuracies)
    avg_balanced_accuracy = sum(balanced_accuracies) / len(balanced_accuracies)
    avg_precision = sum(precisions) / len(precisions)
    avg_recall = sum(recalls) / len(recalls)
    avg_f1 = sum(f1_scores) / len(f1_scores)

    # Print summary of model performance
    print("Average Accuracy:", avg_accuracy)
    print("Average Balanced Accuracy:", avg_balanced_accuracy)
    print("Average Precision:", avg_precision)
    print("Average Recall:", avg_recall)
    print("Average F1 Score:", avg_f1)

    # Compute the false positive rate (FPR), true positive rate (TPR), and thresholds
    fpr, tpr, thresholds = roc_curve(true_labels, predicted_probabilities)

    # Calculate the area under the ROC curve (AUC)
    roc_auc = auc(fpr, tpr)

    return avg_accuracy, avg_balanced_accuracy, avg_precision, avg_recall, avg_f1, fpr, tpr, roc_auc, best_model

def plot_final_roc(roc_results, dataset_name):
    plt.figure(figsize=(12, 8))
    colors = ['blue', 'green', 'orange', 'red', 'purple']
    for idx, (model_name, result) in enumerate(roc_results[dataset_name].items()):
        fpr, tpr, roc_auc = result['fpr'], result['tpr'], result['roc_auc']
        if model_name == 'XGBoost':
            plt.plot(fpr, tpr, lw=4, label=f'{model_name} (AUC = {roc_auc:.2f})', color='purple')
        else:
            plt.plot(fpr, tpr, lw=2, label=f'{model_name} (AUC = {roc_auc:.2f})', alpha=0.2, color=colors[idx])

    plt.plot([0, 1], [0, 1], color='gray', lw=1, linestyle='--')
    plt.xlabel('False Positive Rate', fontsize=14)
    plt.ylabel('True Positive Rate', fontsize=14)
    plt.title(f'ROC Curve for {dataset_name}', fontsize=16)
    plt.legend(loc='lower right', fontsize=12)
    plt.grid(True)
    plt.savefig(os.path.join(output_dir, f'ROC_Curve_{dataset_name}.png'), dpi=300)
    plt.show()

def plot_shap_summary(shap_values, X, model_name, dataset_name):
    plt.figure()
    shap.summary_plot(shap_values, X, plot_type="dot", max_display=10, show=False)
    plt.savefig(os.path.join(output_dir, f'SHAP_Summary_{model_name}_{dataset_name}.png'), dpi=300)
    plt.close()

def plot_shap_waterfall(shap_values, X, model_name, dataset_name):
    # Select an example to explain
    example = shap_values[0]
    plt.figure()
    shap.waterfall_plot(example, show=False)
    plt.savefig(os.path.join(output_dir, f'SHAP_Waterfall_{model_name}_{dataset_name}.png'), dpi=300)
    plt.close()

def plot_feature_importance_bar(feature_importance_df, model_name, dataset_name):
    feature_importance_df_sorted = feature_importance_df.sort_values(by='Importance', ascending=False).head(20)
    colors = ['red' if i < 5 else 'skyblue' for i in range(len(feature_importance_df_sorted))]
    plt.figure(figsize=(10, 8))
    plt.barh(feature_importance_df_sorted['Feature'], feature_importance_df_sorted['Importance'], color=colors)
    plt.xlabel('Importance', fontsize=14)
    plt.ylabel('Feature', fontsize=14)
    plt.title(f'Feature Importances for {model_name} on {dataset_name}', fontsize=16)
    plt.gca().invert_yaxis()
    plt.savefig(os.path.join(output_dir, f'Feature_Importances_{model_name}_{dataset_name}.png'), dpi=300)
    plt.show()

def main():
    datasets_names = ['4-wayAdditive_100feat.txt', '2Additive_2-wayEpi_100feat.txt', '2-wayEpi_100feat.txt', '4-wayHeterogeneous_100feat.txt']
    directory_path = 'data/small_datasets/'
    
    # Iterate over each dataset
    for dataset_name in datasets_names:
        # Construct the full path to the dataset file
        file_path = os.path.join(directory_path, dataset_name)

        # Load the dataset into a DataFrame
        df = pd.read_csv(file_path, delimiter='\t')

        # Print the dataset name
        print(f"Training models for dataset: {dataset_name}")

        # Split data into features and target
        X = df.drop(columns=['Class'])
        y = df['Class']

        # Apply MultiSURF for feature selection
        if do_multisurf:
            msurf = MultiSURF(n_jobs=njobs)
            msurf.fit(X.values, y.values)
            important_feature_indices = msurf.top_features_[:20]  # Select top 20 features, adjust as needed
            X = X.iloc[:, important_feature_indices]
            
               # Save selected features
        selected_features_file = os.path.join(output_dir, f'Selected_Features_{dataset_name}.csv')
        pd.DataFrame(X.columns, columns=["Selected Features"]).to_csv(selected_features_file, index=False)
        print(f"Selected features for {dataset_name} saved to {selected_features_file}")

        # Print selected features
        print("Selected features:", X.columns)

        # Define parameter grids for each model
        param_grids = {
            'Decision Tree': {
                'max_depth': [None, 10, 20, 30],
                'min_samples_split': [2, 5, 10]
            },
            'Random Forest': {
                'n_estimators': [100, 200, 300],
                'max_depth': [None, 10, 20, 30],
                'min_samples_split': [2, 5, 10]
            },
            'SVM': {
                'svc__C': [0.1, 1, 10],
                'svc__kernel': ['linear', 'rbf'],
                'svc__gamma': ['scale', 'auto']
            },
            'Logistic Regression': {
                'C': [0.01, 0.1, 1, 10, 100],
                'solver': ['lbfgs', 'liblinear']
            },
            'XGBoost': {
                'n_estimators': [100, 200, 300],
                'learning_rate': [0.01, 0.1, 0.2],
                'max_depth': [3, 5, 7]
            }
        }

        # Initialize models
        models = {
            'Decision Tree': DecisionTreeClassifier(),
            'Random Forest': RandomForestClassifier(),
            'SVM': Pipeline([
                ('scaler', StandardScaler()),
                ('svc', SVC(probability=True))
            ]),
            'Logistic Regression': LogisticRegression(),
            'XGBoost': XGBClassifier(use_label_encoder=False, eval_metric='logloss')
        }

        # Initialize dictionary to store ROC results for the current dataset
        roc_results[dataset_name] = {}

        # Train and evaluate each model
        for model_name, model in models.items():
            print(f"Evaluating {model_name} on {dataset_name}...")
            param_grid = param_grids.get(model_name, {})
            avg_accuracy, avg_balanced_accuracy, avg_precision, avg_recall, avg_f1, fpr, tpr, roc_auc, best_model = evaluate_model(model, param_grid, X, y, n_splits=5)
            print(f"ROC AUC for {model_name} on {dataset_name}: {roc_auc}")

            # Store ROC results for final plot
            roc_results[dataset_name][model_name] = {'fpr': fpr, 'tpr': tpr, 'roc_auc': roc_auc}

            # SHAP values for XGBoost only
            if model_name == 'XGBoost':
                explainer = shap.Explainer(best_model, X)
                shap_values = explainer(X)
                plot_shap_summary(shap_values, X, model_name, dataset_name)
                plot_shap_waterfall(shap_values, X, model_name, dataset_name)
                
                # Calculate permutation feature importance for XGBoost
                result = permutation_importance(best_model, X, y, n_repeats=10, random_state=42, n_jobs=-1)
                feature_importances = result.importances_mean
                feature_importance_df = pd.DataFrame({'Feature': X.columns, 'Importance': feature_importances})
                feature_importance_df = feature_importance_df.sort_values(by='Importance', ascending=False)
                feature_importance_df.to_csv(os.path.join(output_dir, f'Feature_Importances_{model_name}_{dataset_name}.csv'), index=False)
                print(f"Feature Importances for {model_name}:\n", feature_importance_df.head())

                # Plot feature importance bar plot
                plot_feature_importance_bar(feature_importance_df, model_name, dataset_name)

        # Plot and save ROC curves for the current dataset
        plot_final_roc(roc_results, dataset_name)

if __name__ == "__main__":
    main()
